Materia: Análisis Numérico

Alumno: Chavez Matias David

In [71]:
from IPython.display import display, Markdown, Latex
import random
import numpy as np
from numpy import identity, matmul
from operator import itemgetter

#### Ejercicio 7.

Escribir un código para estimar la norma infnito de una matriz, usar la fórmula cerrada. 

$\qquad$ $$\vert\vert A \vert\vert_\infty = \max_{1 \leq i \leq n} \sum_{j=1}^{n} \vert a_{ij} \vert$$

In [72]:
def normaInfinitoCerrada(matriz):
    res = []
    cant_filas = len(matriz)
    for i in range(cant_filas):
        res.append(sum(map(abs, matriz[i])))

    return max(res)

¿Cómo sería un código si no tenemos una fórmula cerrada? Comparar.

$\qquad$ Si no se puede utilizar la fórmula cerrada, entonces se puede aproximar un valor utilizando la siguiente igualdad:

$\qquad$ $$\vert\vert A \vert\vert_\infty = \max_{\vert\vert x \vert\vert_\infty = 1} \vert\vert A x \vert\vert_\infty $$ siendo $x$ un vector n-dimensional con norma infinito igual a 1.

$\qquad$ Vamos probando con diferentes cantidades de vectores que cumplan esa condición, guardándonos el máximo valor tras cada iteración. De manera de validar la convergencia.


In [73]:
def generacionVectoresAleatorios(tamanio_matriz):
    vector_x = np.random.uniform(-1, 1, tamanio_matriz)
    # Cargamos un 1 para que la norma infinito resulte 1, ya que todos seran menores o iguales a 1 (en modulo)
    vector_x[random.randint(0, tamanio_matriz - 1)] = 1
    return vector_x

def normaInfinitoAproximada(matriz, pasadas):
    max_norma = 0
    for r in range(pasadas + 1):
        cant_filas = len(matriz)
        vec_x = generacionVectoresAleatorios(cant_filas)
        # Multiplicacion A . x
        vec_b = np.dot(matriz, vec_x)
        norma_infinito = max(vec_b)
        max_norma = max(max_norma, norma_infinito)
    
    return max_norma

$\qquad$ Calculamos por ejemplo
$$\begin{bmatrix} 3 & 5 & 7 \\ 2 & 6 & 4 \\ 0 & 2 & 8 \end{bmatrix} $$

In [74]:
matriz = [[3, 5, 7], [2, 6, 4], [0, 2, 8]]
res_cerrada = normaInfinitoCerrada(matriz)

intentos = [10,50,100,500,1000,10000, 100000]
res_intentos = []
for i in range(len(intentos)):
    res_intentos.append(normaInfinitoAproximada(matriz, intentos[i]))

display(Markdown('''
| n  | Sumas realizadas |
|----|----              |
| %s | %d               |
| %s | %.10f               |
| %s | %.10f               |
| %s | %.10f               |
| %s | %.10f               |
| %s | %.10f               |
| %s | %.10f               |
| %s | %.10f               |
''' % ("Cerrada",res_cerrada
      ,"10"     ,res_intentos[0]
      ,"50"     ,res_intentos[1]
      ,"100"    ,res_intentos[2]
      ,"500"    ,res_intentos[3]
      ,"1000"   ,res_intentos[4]
      ,"10000"  ,res_intentos[5]
      ,"100000" ,res_intentos[6]
)
))


| n  | Sumas realizadas |
|----|----              |
| Cerrada | 15               |
| 10 | 13.5314655404               |
| 50 | 12.6244351565               |
| 100 | 13.3797407345               |
| 500 | 14.3088714366               |
| 1000 | 14.6702696714               |
| 10000 | 14.9384490185               |
| 100000 | 14.9694537385               |


$\qquad$ Como se puede apreciar, a medida que aumentamos la cantidad de vectores, el resultado converge hacia el valor de la fórmula cerrada.

#### Ejercicio 11
Escribir un código para resolver el sistema matricial $Ax = b$ donde A es una matriz que es una permutación de una matriz triangular superior.

$\qquad$ Siguiendo el esquema de la matriz indicada, podemos establecer que para calcular cada valor de $x$ vale que:

$\qquad$ $$\forall i \in \{n, n-1,\ldots,1\} \quad x_i = \frac{b_i - \sum_{j=i}^{n} a_{ij}x_j}{a_{ij}}$$

$\qquad$ Es decir, comenzamos a despejar desde abajo hacia arriba. Por cada fila que recorremos, empezamos desde la diagonal, ya que hacia la izquierda hay todos ceros.

$\qquad$ Dado $p = [ p_1, p_2, \ldots, p_n ] $ vector de permutacion de filas A, podemos calcular respetando el orden de la siguiente manera:

$\qquad$ $$\forall i \in \{n, n-1,\ldots,1\} \quad x_i = \frac{b_{p[i]} - \sum_{j=i}^{n} a_{p[i], j}x_j}{a_{p[i], i}}$$

$\qquad$ Hay que tener en cuenta que la pertumación afecta tambien al vector $b$

In [75]:
def resolverMatrizTriangularSuperior(matriz, b, p):
    n = len(matriz)
    res = [None for i in range(n)]

    # primer elemento:
    res[n - 1] = b[p[n - 1]] / matriz[p[n - 1]][n - 1]

    # Recorremos la matriz desde abajo hacia arriba, y de izquierda a derecha, comenzando desde la diagonal + 1.
    for i in range(n - 2, -1, -1):
        acum = 0
        for j in range(i + 1, n):
            acum = acum + matriz[p[i]][j] * res[j]

        res[i] = (b[p[i]] - acum) / matriz[p[i]][i]
        # Validación para que el resultado no me dé -0
        if res[i] == -0:
            res[i] = 0.0

    return res
    

#### Ejercicio 14
Usar el proceso de eleminación de Gauss Escalado para encontrar la descomposición PA=LU (comparar con Python) y resolver en cada uno de los casos.

$$\begin{bmatrix} -1 & 1 & -4 \\ 2 & 2 & 0 \\ 4 & 4 & 2 \end{bmatrix} \begin{bmatrix} x_1 \\ x_2 \\ x_3 \end{bmatrix} = \begin{bmatrix} 0 \\ 1 \\ \frac{1}{2} \end{bmatrix} $$

$$\begin{bmatrix} 1 & 6 & 0 \\ 2 & 1 & 0 \\ 0 & 2 & 1 \end{bmatrix} \begin{bmatrix} x_1 \\ x_2 \\ x_3 \end{bmatrix} = \begin{bmatrix} 3 \\ 1 \\ 1 \end{bmatrix} $$

In [76]:
def solve_pivot(matriz, n, j, s, p):
    radios = [(i, p[i], abs(matriz[i][j]) / s[i]) for i in p[j:]]
    pivot = max(radios, key=itemgetter(2))[1]
    p[j], p[pivot] = p[pivot], p[j]
    return pivot


def solve_gauss(matriz, b):
    n = len(matriz)
    s = [max(map(abs, row)) for row in matriz]
    p = [i for i in range(n)]

    # recorro por columna, calculo pivot, aplico eliminacion
    for j in range(n - 1):
        #  para la columna j, obtengo que fila es pivot
        pivot = solve_pivot(matriz, n, j, s, p)

        # una vez tengo el pivot, tengo que aplicar la eliminacion para las filas diferentes al pivot
        for i in range(j + 1, n):
            factor = matriz[p[i]][j] / matriz[pivot][j]
            b[p[i]] = b[p[i]] - factor * b[pivot]
            for k in range(n):
                matriz[p[i]][k] = matriz[p[i]][k] - factor * matriz[pivot][k]
            matriz[p[i]][j] = factor
    # Armamos L, U y P segun A
    mp = [[0 for _ in range(n)] for _ in range(n)]
    ml = [[0 for _ in range(n)] for _ in range(n)]
    mu = [[0 for _ in range(n)] for _ in range(n)]

    for i in range(n):
        ml[i][i] = 1
        mp[i][p[i]] = 1
        for j in range(0, i):
            ml[i][j] = matriz[p[i]][j]
        for j in range(i, n):
            mu[i][j] = matriz[p[i]][j]

    return matriz, mp, ml, mu, b, p


$\qquad$ Primer Sistema

$$\begin{bmatrix} -1 & 1 & -4 \\ 2 & 2 & 0 \\ 4 & 4 & 2 \end{bmatrix} \begin{bmatrix} x_1 \\ x_2 \\ x_3 \end{bmatrix} = \begin{bmatrix} 0 \\ 1 \\ \frac{1}{2} \end{bmatrix} $$
$$ \begin{bmatrix} x_1 \\ x_2 \\ x_3 \end{bmatrix} = \begin{bmatrix} \frac{5}{4} \\ -\frac{3}{4} \\ -\frac{1}{4} \end{bmatrix} $$

In [77]:
matriz_1 = [[-1, 1, -4], [2, 2, 0], [3, 3, 2]]
b = [0, 1, 0.5]
p = []
a, mp, ml, mu, b, p = solve_gauss(matriz_1, b)

print("A => \n", np.matrix(a))
print("B => \n", np.matrix(b))
print("P => \n", np.matrix(mp))
print("L => \n", np.matrix(ml))
print("U => \n", np.matrix(mu))
print("")
print("x => ", resolverMatrizTriangularSuperior(a, b, p))

A => 
 [[-0.5  2.  -4. ]
 [ 2.   2.   0. ]
 [ 1.5  0.   2. ]]
B => 
 [[ 0.5  1.  -1. ]]
P => 
 [[0 1 0]
 [1 0 0]
 [0 0 1]]
L => 
 [[ 1.   0.   0. ]
 [-0.5  1.   0. ]
 [ 1.5  0.   1. ]]
U => 
 [[ 2.  2.  0.]
 [ 0.  2. -4.]
 [ 0.  0.  2.]]

x =>  [1.25, -0.75, -0.5]


$\qquad$ Segundo Sistema

$$\begin{bmatrix} 1 & 6 & 0 \\ 2 & 1 & 0 \\ 0 & 2 & 1 \end{bmatrix} \begin{bmatrix} x_1 \\ x_2 \\ x_3 \end{bmatrix} = \begin{bmatrix} 3 \\ 1 \\ 1 \end{bmatrix} $$

$$ \begin{bmatrix} x_1 \\ x_2 \\ x_3 \end{bmatrix} = \begin{bmatrix} \frac{3}{11} \\ \frac{5}{11} \\ \frac{1}{11} \end{bmatrix} $$

In [78]:
matriz_2 = [[1, 6, 0], [2, 1, 0], [0, 2, 1]]
b = [3, 1, 1]
p = []
a, mp, ml, mu, b, p = solve_gauss(matriz_2, b)

print("A => \n", np.matrix(a))
print("B => \n", np.matrix(b))
print("p => \n", np.matrix(p))
print("P => \n", np.matrix(mp))
print("L => \n", np.matrix(ml))
print("U => \n", np.matrix(mu))

print("x => ", resolverMatrizTriangularSuperior(a, b, p))

A => 
 [[ 0.5   2.75 -2.75]
 [ 2.    1.    0.  ]
 [ 0.    2.    1.  ]]
B => 
 [[-0.25  1.    1.  ]]
p => 
 [[1 2 0]]
P => 
 [[0 1 0]
 [0 0 1]
 [1 0 0]]
L => 
 [[1.   0.   0.  ]
 [0.   1.   0.  ]
 [0.5  2.75 1.  ]]
U => 
 [[ 2.    1.    0.  ]
 [ 0.    2.    1.  ]
 [ 0.    0.   -2.75]]
x =>  [0.2727272727272727, 0.45454545454545453, 0.09090909090909091]


#### Ejercicio 15
Calcular los primeros 100 términos del método de Richardson para el sistema

$$\begin{bmatrix} 1 & \frac{1}{2} & \frac{1}{3} \\ \frac{1}{3} & 1 & \frac{1}{2} \\ \frac{1}{2} & \frac{1}{3} & 1 \end{bmatrix} \begin{bmatrix} x_1 \\ x_2 \\ x_3 \end{bmatrix} = \begin{bmatrix} \frac{11}{18} \\ \frac{11}{18} \\ \frac{11}{18} \end{bmatrix} $$

$\qquad$ Resolviendo el sistema se llega a que el resultado esperado sea $x = [ \frac{1}{3},\frac{1}{3}, \frac{1}{3} ]$

In [79]:
def solve_richardson(A, b, cant_terminos):

    mR = identity(len(A)) - A
    x = b

    for i in range(cant_terminos):
        x = matmul(mR, x) + b
        print('Iteración n°', i, '=>  x = ', x)

A = [[1, 1 / 2, 1 / 3],
            [1 / 3, 1, 1 / 2],
            [1 / 2, 1 / 3, 1]]

b = [11 / 18, 11 / 18, 11 / 18]

solve_richardson(A, b, 100)

Iteración n° 0 =>  x =  [0.10185185 0.10185185 0.10185185]
Iteración n° 1 =>  x =  [0.52623457 0.52623457 0.52623457]
Iteración n° 2 =>  x =  [0.1725823 0.1725823 0.1725823]
Iteración n° 3 =>  x =  [0.46729252 0.46729252 0.46729252]
Iteración n° 4 =>  x =  [0.22170067 0.22170067 0.22170067]
Iteración n° 5 =>  x =  [0.42636055 0.42636055 0.42636055]
Iteración n° 6 =>  x =  [0.25581065 0.25581065 0.25581065]
Iteración n° 7 =>  x =  [0.39793557 0.39793557 0.39793557]
Iteración n° 8 =>  x =  [0.27949814 0.27949814 0.27949814]
Iteración n° 9 =>  x =  [0.378196 0.378196 0.378196]
Iteración n° 10 =>  x =  [0.29594778 0.29594778 0.29594778]
Iteración n° 11 =>  x =  [0.36448796 0.36448796 0.36448796]
Iteración n° 12 =>  x =  [0.30737114 0.30737114 0.30737114]
Iteración n° 13 =>  x =  [0.35496849 0.35496849 0.35496849]
Iteración n° 14 =>  x =  [0.31530404 0.31530404 0.31530404]
Iteración n° 15 =>  x =  [0.34835775 0.34835775 0.34835775]
Iteración n° 16 =>  x =  [0.32081299 0.32081299 0.32081299]

#### Ejercicio 16
Escribir un algoritmo para calcular los primeros M pasos del método de Jacobi, y Gauss-Seidel

<div  style="text-align: center;">
<img src="img/pseudo_jacobi.png" width="385px">
<img src="img/pseudo_gauss_seidel.jpg" width="500px">
</div>

In [80]:
def solve_jacobi(A, b, r, x):
    n = len(A)
    acum = 0
    temp = [0.0 for _ in range(len(A))]

    print(f"{1} \t {x[0]:.6f} \t {x[1]:.6f} \t {x[2]:.6f}  ")
    for k in range(2, r):
        x = temp
        for i in range(0, n):
            acum = 0
            for j in range(0, n):
                if j != i:
                    acum += A[i][j] * x[j]
                    # print("acum: ", acum)
                temp[i] = (b[i] - acum) / A[i][i]

        print(f"{k} \t {x[0]:.6f} \t {x[1]:.6f} \t {x[2]:.6f}  ")

def solve_gauss_seidel(A, b, r, x):
    n = len(A)
    acum = 0
    temp = [0.0 for _ in range(len(A))]

    print(f"{1} \t {x[0]:.6f} \t {x[1]:.6f} \t {x[2]:.6f}  ")

    for k in range(0, r):
        for i in range(0, n):
            acum = 0
            for j in range(0, n):
                if j != i:
                    acum += A[i][j] * x[j]
                    x[i] = (b[i] - acum) / A[i][i]

        print(f"{k} \t {x[0]:.6f} \t {x[1]:.6f} \t {x[2]:.6f}  ")

matriz_A = [[1, 1 / 2, 1 / 3],
            [1 / 3, 1, 1 / 2],
            [1 / 2, 1 / 3, 1]]

vector_B = [11 / 18, 11 / 18, 11 / 18]

x = [0.0 for _ in range(len(A))]

print("Jacobi solver")
solve_jacobi(matriz_A, vector_B, 10, x)
print("")
print("Gauss-Seidel solver")
solve_gauss_seidel(matriz_A, vector_B, 10, x)

Jacobi solver
1 	 0.000000 	 0.000000 	 0.000000  
2 	 0.611111 	 0.407407 	 0.169753  
3 	 0.350823 	 0.409294 	 0.299268  
4 	 0.306708 	 0.359241 	 0.338010  
5 	 0.318821 	 0.335833 	 0.339757  
6 	 0.329943 	 0.331252 	 0.335722  
7 	 0.333578 	 0.332057 	 0.333637  
8 	 0.333870 	 0.333003 	 0.333175  
9 	 0.333551 	 0.333340 	 0.333222  

Gauss-Seidel solver
1 	 0.000000 	 0.000000 	 0.000000  
0 	 0.611111 	 0.407407 	 0.169753  
1 	 0.350823 	 0.409294 	 0.299268  
2 	 0.306708 	 0.359241 	 0.338010  
3 	 0.318821 	 0.335833 	 0.339757  
4 	 0.329943 	 0.331252 	 0.335722  
5 	 0.333578 	 0.332057 	 0.333637  
6 	 0.333870 	 0.333003 	 0.333175  
7 	 0.333551 	 0.333340 	 0.333222  
8 	 0.333367 	 0.333378 	 0.333302  
9 	 0.333322 	 0.333353 	 0.333333  


$\qquad$ Utilizando el mismo sistema que para el algoritmo de Richardson, vemos que ambos metodos convergen a la solución correcta.

#### Ejercicio 17
Para el siguiente sistema mostrar que tanto Gauss Seidel como Jacobi convergen para cualquier valor inicial. 
Usar el ítem anterior para estimar la solución.

$$\begin{bmatrix} 2 & -1 & 0 \\ 1 & 6 & -2 \\ 4 & -3 & 8 \end{bmatrix} \begin{bmatrix} x_1 \\ x_2 \\ x_3 \end{bmatrix} = \begin{bmatrix} 2 \\ -4 \\ 5 \end{bmatrix} $$

$$ \begin{bmatrix} x_1 \\ x_2 \\ x_3 \end{bmatrix} = \begin{bmatrix} \frac{31}{50} \\ -\frac{19}{25} \\ -\frac{3}{100} \end{bmatrix} = \begin{bmatrix} 0.62 \\ -0.76 \\ -0.03 \end{bmatrix} $$

In [81]:
A = [[2,-1,0],[1,6,-2],[4,-3,8]]
b = [2,-4,5]

x = np.random.uniform(-1, 1, len(A))
print("Jacobi solver => x: ", x)
solve_jacobi(A, b, 10, x)
print("")
print("Gauss-Seidel solver => x: ", x)
x = np.random.uniform(-1, 1, len(A))
solve_gauss_seidel(A, b, 10, x)


Jacobi solver => x:  [-0.11951146  0.82486172  0.11046228]
1 	 -0.119511 	 0.824862 	 0.110462  
2 	 1.000000 	 -0.833333 	 -0.187500  
3 	 0.583333 	 -0.826389 	 0.023438  
4 	 0.586806 	 -0.756655 	 0.047852  
5 	 0.621672 	 -0.754328 	 0.031291  
6 	 0.622836 	 -0.760042 	 0.028566  
7 	 0.619979 	 -0.760474 	 0.029833  
8 	 0.619763 	 -0.760016 	 0.030113  
9 	 0.619992 	 -0.759961 	 0.030019  

Gauss-Seidel solver => x:  [-0.11951146  0.82486172  0.11046228]
1 	 0.233920 	 -0.820433 	 0.649955  
0 	 0.589784 	 -0.548312 	 0.124491  
1 	 0.725844 	 -0.746144 	 -0.017726  
2 	 0.626928 	 -0.777063 	 0.020137  
3 	 0.611468 	 -0.761866 	 0.033566  
4 	 0.619067 	 -0.758656 	 0.030970  
5 	 0.620672 	 -0.759789 	 0.029743  
6 	 0.620106 	 -0.760103 	 0.029908  
7 	 0.619948 	 -0.760022 	 0.030018  
8 	 0.619989 	 -0.759992 	 0.030008  
9 	 0.620004 	 -0.759998 	 0.029999  


$\qquad$ Como se puede observar, tomando vectores $x$ diferentes, se converge al valor esperado.